In [1]:
#%% ----------------------------------- Package Imports
import pandas as pd
from numpy import nan
import dash
import dash_table
from fredapi import Fred
fred = Fred(api_key='287dcf1a4ec41e9f44645fb74180eb42')
import pandas as pd
import numpy as num
from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox, Layout
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
plt.style.use('seaborn')
%config InlineBackend.figure_format = 'svg'
from __future__ import print_function
from bqplot import topo_load
from bqplot.interacts import panzoom
import datetime as dt

In [ ]:
def pv(cash_flow, period, day_count, cost_of_capital):
    #day_count is meant to proxy the length of the "period" the cash flow is received in 
    v = cash_flow/((1+cost_of_capital)**(period*day_count))
    return v

In [8]:
from ipywidgets import interactive,interact, HBox, Layout,VBox
#function to plot the different curves
def plot_function(u=1,v=2,w=3,x=4,y=5,z=6):
    time=np.arange(0,1,0.01)
    df=pd.DataFrame({"Y1":np.sin(time*u*2*np.pi),"y2":np.sin(time*v*2*np.pi),"y3":np.sin(time*w*2*np.pi),
                    "y4":np.sin(time*x*2*np.pi),"y5":np.sin(time*y*2*np.pi),"y6":np.sin(time*z*2*np.pi)})
    df.plot()
widget=interactive(plot_function,u=1,v=2,w=3,x=4,y=5,z=6)
controls = HBox(widget.children[:-1], layout = Layout(flex_flow='row wrap'))
output = widget.children[-1]
display(VBox([controls, output]))

In [2]:
df = fred.search('US rates')
USrates = df.loc[df['frequency']=='Monthly']
#df['title'].T
#df = df['title'].T
#df.columns
#df.loc(df['frequency_short']=='M')
#df = df.loc[df['frequency']=='Monthly'].T

In [3]:
import datetime
dates = [datetime.date(2015,i,1) for i in range(1,13)]
options = [(i.strftime('%b'), i) for i in dates]
widgets.SelectionRangeSlider(
    options=options,
    index=(0,11),
    description='Months (2015)',
    disabled=False
)

SelectionRangeSlider(description='Months (2015)', index=(0, 11), options=(('Jan', datetime.date(2015, 1, 1)), …

In [6]:
x = widgets.Dropdown(options=USrates['title'].tolist(),description='FRED Dataseries:')
x.layout.width = '800px'

y = widgets.IntSlider(
    value=3,
    min=0,
    step=1,
    description='RSI Window:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

a = widgets.IntSlider(
    value=90,
    min=0,
    max=100,
    step=1,
    description='Buy Signal:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

b = widgets.IntSlider(
    value=10,
    min=0,
    max=100,
    step=1,
    description='Sell Signal:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

c = widgets.IntSlider(
    value=50,
    min=0,
    max=100,
    step=1,
    description='Close Buy:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

d = widgets.IntSlider(
    value=50,
    min=0,
    max=100,
    step=1,
    description='Close Sell:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')
#display(x)

def g(x,y,a,b,c,d):
    
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.collections import LineCollection
    from matplotlib.colors import ListedColormap, BoundaryNorm
    
    plot_series = USrates.loc[USrates['title']==x].index[0]

    def generate_RSI_stats(df, series,RSI_decay=3,buy_signal=90, sel_signal=10, buy_exit=50,sel_exit=50):
        df = pd.DataFrame(data = fred.get_series(series),columns =[series])
        dff = 'diff_'+series
        dn = series+'_dn'
        up = series+'_up'
        df[dff] = df[series].diff()
        df[up] = df[dff]
        df.loc[df[dff] < 0, up] = 0
        df[up] = (df[up]).ewm(alpha=(1/RSI_decay)).mean()
        df[dn] = df[dff]
        df.loc[df[dff] > 0, dn] = 0
        df[dn] = (df[dn]).ewm(alpha=(1/RSI_decay)).mean()
        df[series+'_RSI_signal'] = 100-100/(1-df[up]/df[dn])
                
        buy_sel_signal = str(series+'_buy_sel_signal')
        # Calculate Buy Signal
        #df['buy_signal'] = nan
        #df['buy_signal'] = 0
        df.loc[df[series+'_RSI_signal'] > buy_signal, 'buy_signal'] = 1
        df.loc[df[series+'_RSI_signal'] <= buy_exit, 'buy_signal'] = 0
        df['buy_signal'] = df['buy_signal'].ffill()
        df.loc[(df['buy_signal'].shift() == 1) & (df['buy_signal'] == 0), 'buy_signal'] = -1
        
        # Calculate Sell Signal
        df.loc[df[series+'_RSI_signal'] < sel_signal, 'sel_signal'] = 1
        df.loc[df[series+'_RSI_signal'] >= sel_exit, 'sel_signal'] = 0
        df['sel_signal'] = df['sel_signal'].ffill()
        df.loc[(df['sel_signal'].shift() == 1) & (df['sel_signal'] == 0), 'sel_signal'] = -1
        
        # Combine
        df[buy_sel_signal] = df['buy_signal'] - df['sel_signal']
        # Cleanup
        df[buy_sel_signal] = df[buy_sel_signal].fillna(0)
        #df.drop(['buy_signal', 'sel_signal'], axis=1, inplace=True)
        
        df['MTM']= df[series+'_buy_sel_signal']*df[dff]
        df = df[[series,series+'_RSI_signal','MTM']]
        return df
    
    plots = generate_RSI_stats(df = fred.get_series(plot_series), series=plot_series,RSI_decay=y,
                               buy_signal=a, sel_signal=b, buy_exit=c,sel_exit=d).plot(subplots=True)
    plt.show()
        
interact(g,x=x,y=y,a=a,b=b,c=c,d=d);


interactive(children=(Dropdown(description='FRED Dataseries:', layout=Layout(width='800px'), options=('10-Year…

In [35]:
x = widgets.Dropdown(options=USrates['title'].tolist())
x.layout.width = '900px'

y = widgets.IntSlider(
    value=3,
    min=0,
    step=1,
    description='RSI Window:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

a = widgets.IntSlider(
    value=90,
    min=0,
    max=100,
    step=1,
    description='Buy Signal:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

b = widgets.IntSlider(
    value=10,
    min=0,
    max=100,
    step=1,
    description='Sell Signal:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

c = widgets.IntSlider(
    value=50,
    min=0,
    max=100,
    step=1,
    description='Close Buy:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

d = widgets.IntSlider(
    value=50,
    min=0,
    max=100,
    step=1,
    description='Close Sell:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

def g(x,y,a,b,c,d):
    
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.collections import LineCollection
    from matplotlib.colors import ListedColormap, BoundaryNorm
    
    plot_series = USrates.loc[USrates['title']==x].index[0]

    def generate_RSI_stats(df, series,RSI_decay=3,buy_signal=90, sel_signal=10, buy_exit=50,sel_exit=50):
        df = pd.DataFrame(data = fred.get_series(series),columns =[series])
        dff = 'diff_'+series
        dn = series+'_dn'
        up = series+'_up'
        df[dff] = df[series].diff()
        df[up] = df[dff]
        df.loc[df[dff] < 0, up] = 0
        df[up] = (df[up]).ewm(alpha=(1/RSI_decay)).mean()
        df[dn] = df[dff]
        df.loc[df[dff] > 0, dn] = 0
        df[dn] = (df[dn]).ewm(alpha=(1/RSI_decay)).mean()
        df[series+'_RSI_signal'] = 100-100/(1-df[up]/df[dn])
                
        buy_sel_signal = str(series+'_buy_sel_signal')
        # Calculate Buy Signal
        #df['buy_signal'] = nan
        #df['buy_signal'] = 0
        df.loc[df[series+'_RSI_signal'] > buy_signal, 'buy_signal'] = 1
        df.loc[df[series+'_RSI_signal'] <= buy_exit, 'buy_signal'] = 0
        df['buy_signal'] = df['buy_signal'].ffill()
        df.loc[(df['buy_signal'].shift() == 1) & (df['buy_signal'] == 0), 'buy_signal'] = -1
        
        # Calculate Sell Signal
        df.loc[df[series+'_RSI_signal'] < sel_signal, 'sel_signal'] = 1
        df.loc[df[series+'_RSI_signal'] >= sel_exit, 'sel_signal'] = 0
        df['sel_signal'] = df['sel_signal'].ffill()
        df.loc[(df['sel_signal'].shift() == 1) & (df['sel_signal'] == 0), 'sel_signal'] = -1
        
        # Combine
        df[buy_sel_signal] = df['buy_signal'] - df['sel_signal']
        # Cleanup
        df[buy_sel_signal] = df[buy_sel_signal].fillna(0)
        #df.drop(['buy_signal', 'sel_signal'], axis=1, inplace=True)
        
        df['MTM']= df[series+'_buy_sel_signal']*df[dff]
        df = df[series+'_RSI_signal']
        return df
    
    #plots = generate_RSI_stats(df = fred.get_series(plot_series), series=plot_series,RSI_decay=y,
     #                          buy_signal=a, sel_signal=b, buy_exit=c,sel_exit=d)

    data = generate_RSI_stats(df = fred.get_series(plot_series), series=plot_series,RSI_decay=y,
                               buy_signal=a, sel_signal=b, buy_exit=c,sel_exit=d)
    
    trace = go.Scatter(x=list(data[1].tolist()))
    
    layout = dict(
        title='Time series with range slider and selectors',
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                        label='1m',
                         step='month',
                         stepmode='backward'),
                        dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(count=1,
                        label='YTD',
                        step='year',
                        stepmode='todate'),
                    dict(count=1,
                        label='1y',
                        step='year',
                        stepmode='backward'),
                    dict(step='all')
                ])
            ),
            rangeslider=dict(
                visible = True
            ),
            type='date'
        )
    )

    fig = dict(data=data, layout=layout)
    py.iplot(fig)
    
interact(g,x=x,y=y,a=a,b=b,c=c,d=d);


interactive(children=(Dropdown(description='x', layout=Layout(width='900px'), options=('10-Year Treasury Const…

In [7]:
@interact(cash_flow=widgets.FloatText(
    value=100,
    description='Initial Cash Flow:',
    disabled=False
),period=widgets.FloatText(
    value=1,
    description='Discount Periods:',
    disabled=False
), day_count=widgets.FloatText(
    value=1,
    description='Day Count:',
    disabled=False
), cost_of_capital=widgets.FloatText(
    value=.07,
    description='Cost of Capital:',
    disabled=False
))
def pv(cash_flow, period, day_count, cost_of_capital):
    #day_count is meant to proxy the length of the "period" the cash flow is received in 
    v = cash_flow/((1+cost_of_capital)**(period*day_count))
    return v

interactive(children=(FloatText(value=100.0, description='Initial Cash Flow:'), FloatText(value=1.0, descripti…

In [25]:
import plotly.plotly as py
import plotly.graph_objs as go 

from datetime import datetime
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')
df.columns = [col.replace('AAPL.', '') for col in df.columns]

trace = go.Scatter(x=list(df.Date),
                   y=list(df.High))

data = [trace]
layout = dict(
    title='Time series with range slider and selectors',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(count=1,
                    label='YTD',
                    step='year',
                    stepmode='todate'),
                dict(count=1,
                    label='1y',
                    step='year',
                    stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)

fig = dict(data=data, layout=layout)
py.iplot(fig)

In [43]:
import plotly.plotly as py
import plotly.graph_objs as go

df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/school_earnings.csv")


data = [go.Bar(x=df.School,y=df.Gap)]

py.iplot(data, filename='jupyter-basic_bar')

In [52]:
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import numpy as np
import plotly.plotly as py

#py.sign_in('DemoAccount', 'lr1c37zw81')

n = 50
x, y, z, s, ew = np.random.rand(5, n)
c, ec = np.random.rand(2, n, 4)
area_scale, width_scale = 500, 5

fig, ax = plt.subplots()
sc = ax.scatter(x, y, c=c,
                s=np.square(s)*area_scale,
                edgecolor=ec,
                linewidth=ew*width_scale)
ax.grid()

#plot_url = 
plotly.offline.iplot_mpl(fig)

<module 'plotly.offline' from 'C:\\Users\\Carlos Bonilla\\Anaconda3\\lib\\site-packages\\plotly\\offline\\__init__.py'>

In [51]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import os
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go
from app import app

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/mtcars.csv')

if 'DYNO' in os.environ:
    app_name = os.environ['DASH_APP_NAME']
else:
    app_name = 'dash-multiplesubplot'

layout = html.Div([
    html.Div([html.H1("Car Performance Metrics")], style={"textAlign": "center"}),
    html.Div([html.Div([html.Div([html.Span("Scatter Plot x-axis", className="three columns",
                                            style={"width": 150, "padding": 10, "text-align": "right"}),
                                  html.Div(dcc.Dropdown(id="xaxis",
                                                        options=[{'label': "Miles per gallon", 'value': "mpg"},
                                                                 {'label': "Displacement", 'value': "disp"},
                                                                 {'label': "Rear axle ratio", 'value': "drat"},
                                                                 {'label': "Weight (1000 lbs)", 'value': "wt"},
                                                                 {'label': "1/4 mile time", 'value': "qsec"},
                                                                 {'label': "Gross horsepower", 'value': "hp"}],
                                                        value='disp', ), className="three columns",
                                           style={"width": 250, "margin": 0})], className="row"),
                        html.Div([html.Span("Scatter Plot y-axis", className="three columns",
                                            style={"width": 150, "padding": 10, "text-align": "right"}),
                                  html.Div(dcc.Dropdown(id="yaxis",
                                                        options=[{'label': "Miles per gallon", 'value': "mpg"},
                                                                 {'label': "Displacement", 'value': "disp"},
                                                                 {'label': "Rear axle ratio", 'value': "drat"},
                                                                 {'label': "Weight (1000 lbs)", 'value': "wt"},
                                                                 {'label': "1/4 mile time", 'value': "qsec"},
                                                                 {'label': "Gross horsepower", 'value': "hp"}],
                                                        value='mpg', ), className="three columns",
                                           style={"width": 250, "margin": 0})], className="row"),
                        html.Div([html.Span("Bar Plot y-axis", className="three columns",
                                            style={"width": 150, "padding": 10, "text-align": "right"}),
                                  html.Div(dcc.Dropdown(id="barplot-yaxis",
                                                        options=[{'label': "Miles per gallon ", 'value': "mpg"},
                                                                 {'label': "Displacement", 'value': "disp"},
                                                                 {'label': "Rear axle ratio", 'value': "drat"},
                                                                 {'label': "Weight (1000 lbs)", 'value': "wt"},
                                                                 {'label': "1/4 mile time ", 'value': "qsec"},
                                                                 {'label': "Gross horsepower", 'value': "hp"}],
                                                        value='hp', ), className="three columns",
                                           style={"width": 250, "margin": 0})], className="row"), ],
                       style={'width': '48%', 'display': 'inline-block'}, className="six columns"),
              html.Div([html.Div([html.Span("Box Plot x-axis", className="three columns",
                                            style={"text-align": "right", "width": "30%", "padding": 5}),
                                  html.Div(dcc.RadioItems(id="select-value", value='vs',
                                                          options=[{'label': "Transmission", 'value': "am"},
                                                                   {'label': "Engine-Type", 'value': "vs"}],
                                                          labelStyle={'display': 'inline', "padding": 7.5}),
                                           className="three columns",
                                           style={"width": "50%", "margin": 0, "padding": 5})], className="row"),
                        html.Div([html.Span("Box Plot y-axis", className="three columns",
                                            style={"width": 150, "padding": 10, "text-align": "right"}),
                                  html.Div(dcc.Dropdown(id="boxplot-yaxis",
                                                        options=[{'label': "Miles per gallon ", 'value': "mpg"},
                                                                 {'label': "Displacement", 'value': "disp"},
                                                                 {'label': "Rear axle ratio", 'value': "drat"},
                                                                 {'label': "Weight (1000 lbs)", 'value': "wt"},
                                                                 {'label': "1/4 mile time ", 'value': "qsec"},
                                                                 {'label': "Gross horsepower", 'value': "hp"}],
                                                        value='mpg', ), className="three columns",
                                           style={"width": 250, "margin": 0})], className="row")
                        ], style={'width': '50%', 'float': 'right', 'display': 'inline-block', "margin-left": "auto",
                                  "margin-right": "auto", "padding-top": 30}, className="six columns")],
             className="row"),
    html.Div([dcc.Graph(id="my-graph")]),
], className="container")


@app.callback(
    dash.dependencies.Output("my-graph", "figure"),
    [dash.dependencies.Input("xaxis", "value"), dash.dependencies.Input("yaxis", "value"),
     dash.dependencies.Input("boxplot-yaxis", "value"), dash.dependencies.Input("select-value", "value"),
     dash.dependencies.Input("barplot-yaxis", "value")])
def update_graph(selected1, selected2, selected_box_y, selected_box_x, selected_bar_y):
    trace1 = [go.Scatter(x=df[selected1], y=df[selected2], text=df['manufacturer'], mode='markers', opacity=0.8,
                         marker={'size': 10, "color": "#00CC94", }, showlegend=False, )]

    trace2 = [go.Box(x=df[selected_box_x], y=df[selected_box_y],
                     name=f'{"Transmission" if selected_box_x == "am" else "Engine Type"}', showlegend=False,
                     xaxis='x2', yaxis='y2', marker={"color": "#4CEB00"})]
    trace3 = [go.Bar(x=df['manufacturer'], y=df[selected_bar_y], xaxis='x3', yaxis='y3', showlegend=False,
                     marker={"color": "#5DB4F2", "opacity": 0.9})]

    trace = trace1 + trace2 + trace3
    text = {"mpg": "Miles per gallon (mpg)", "disp": "Displacement (cu.in.)", "drat": "Rear axle ratio (drat)",
            "qsec": "1/4 mile time (qsec)", "wt": "Weight (1000 lbs)", "hp": "Gross horsepower (hp)"}
    return {"data": trace,
            "layout": go.Layout(height=850, width=1000, annotations=[
                {"x": 0.07, "y": 1, "xref": "paper", "yref": "paper","font": {"size": 12},
                 "text": f'{text[selected2].title()} vs {text[selected1].title()}', "showarrow": False},
                {"x": 0.85, "y": 1, "xref": "paper", "yref": "paper","showarrow": False, "font": {"size": 12},
                 "text": f'{text[selected_box_y].title()} vs 'f'{"Transmission" if selected_box_x == "am" else "Engine Type"}'},
                {"x": 0.5, "y": 0.45, "xref": "paper", "yref": "paper",
                 "text": f'{text[selected_bar_y].title()} Vs Car Model', "showarrow": False, "font": {"size": 15}}],
                                xaxis={"title": text[selected1].title(), "domain": [0, 0.40], "anchor": 'y', },
                                yaxis={"title": text[selected2].title(), "domain": [0.65, 1], "anchor": 'x'}, xaxis2={
                    "title": f'{"Automatic Transmission   Manual Transmission " if selected_box_x == "am" else "V-shaped Engine   Straight Engine"}',
                    "domain": [0.55, 1], "anchor": 'y2',
                    "showticklabels": False, },
                                yaxis2={"title": text[selected_box_y].title(), "domain": [0.55, 1], "anchor": 'x2'},
                                xaxis3={"tickangle": -38, "domain": [0, 1], "anchor": 'y3'},
                                yaxis3={"title": text[selected_bar_y].title(), "domain": [0, 0.45], "anchor": 'x3'})
            }

ModuleNotFoundError: No module named 'app'

In [ ]:
#Notes
#https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6